In [ ]:
import random
import re
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
import torch
from collections import Counter
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Load advanced models
print("Loading advanced NLP models...")
try:
    nlp = spacy.load("en_core_web_md")  # Medium-sized spaCy model with word vectors
except:
    print("Installing spaCy model...")
    import os
    os.system("python -m spacy download en_core_web_md")
    nlp = spacy.load("en_core_web_md")

# Load a more powerful sentence transformer model
model = SentenceTransformer('paraphrase-mpnet-base-v2')  # More advanced than MiniLM

# Load T5-large model for question generation
t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large')

asked_questions = set()

Loading advanced NLP models...
Installing spaCy model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def extract_key_entities(text):
    """Extract important entities and concepts using spaCy."""
    doc = nlp(text)

    # Extract named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Extract noun chunks (important phrases)
    noun_chunks = [chunk.text for chunk in doc.noun_chunks if not all(token.is_stop for token in chunk)]

    # Extract key words based on dependency parsing
    subjects = [token.text for token in doc if token.dep_ in ('nsubj', 'nsubjpass')]
    objects = [token.text for token in doc if token.dep_ in ('dobj', 'pobj', 'attr')]

    # Get keywords using statistical importance
    keywords = [token.text for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    keyword_freq = Counter(keywords)
    important_keywords = [word for word, count in keyword_freq.most_common(5)]

    return {
        'entities': entities,
        'noun_chunks': noun_chunks,
        'subjects': subjects,
        'objects': objects,
        'keywords': important_keywords
    }

def generate_t5_questions(context, num_questions=3):
    """Generate questions using T5 large model."""
    input_text = "generate questions: " + context

    # Tokenize and generate
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = t5_model.generate(
        input_ids=input_ids,
        max_length=100,
        num_return_sequences=num_questions,
        num_beams=8,
        temperature=1.0,
        top_k=100,
        top_p=0.95,
        do_sample=True,
        early_stopping=True
    )

    # Decode and clean up the generated questions
    questions = []
    for output in outputs:
        question = t5_tokenizer.decode(output, skip_special_tokens=True)
        # T5 sometimes generates "question: " prefix
        question = question.replace("question: ", "").strip()
        if question and question not in questions and question.endswith("?"):
            questions.append(question)

    return questions

def is_definition_sentence(sentence):
    """Check if a sentence contains a definition pattern."""
    definition_patterns = [
        "is defined as", "refers to", "means", "is a", "are", "represents",
        "consists of", "is characterized by", "is known as", "can be described as"
    ]
    return any(pattern in sentence.lower() for pattern in definition_patterns)

def generate_theory_questions(chunk):
    """Generate questions focused on theoretical knowledge and definitions."""
    doc = nlp(chunk)
    questions = []

    # Look for potential definition sentences and key terms
    for sent in doc.sents:
        sent_text = sent.text

        if is_definition_sentence(sent_text):
            # Extract the term being defined
            for noun_chunk in nlp(sent_text).noun_chunks:
                if len(noun_chunk.text.split()) <= 3 and not all(token.is_stop for token in noun_chunk):
                    term = noun_chunk.text
                    theory_questions = [
                        f"What is {term}?",
                        f"Define {term} in your own words.",
                        f"Explain the concept of {term}."
                    ]
                    questions.append(random.choice(theory_questions))
                    break

    # Extract key entities and create questions about them
    key_info = extract_key_entities(chunk)
    entities = [e[0] for e in key_info['entities']]

    if entities:
        for entity in entities[:2]:  # Limit to avoid too many similar questions
            if len(entity.split()) <= 3:  # Keep it concise
                theory_questions = [
                    f"What is the significance of {entity}?",
                    f"Describe {entity} according to the text.",
                    f"Explain the role of {entity} in this context."
                ]
                questions.append(random.choice(theory_questions))

    # Check for processes or methods in the text
    process_indicators = ["process", "method", "procedure", "technique", "approach", "steps"]
    for indicator in process_indicators:
        if indicator in chunk.lower():
            questions.append(f"Describe the {indicator} mentioned in the text.")
            break

    return questions[:2]  # Limit to 2 theory questions per chunk

def generate_advanced_questions(chapter_text):
    """Generate high-quality questions with both conceptual and theoretical focus."""
    sentences = sent_tokenize(chapter_text)
    questions = []

    # Extract global concepts and themes first
    doc = nlp(chapter_text)
    global_entities = extract_key_entities(chapter_text)
    global_concepts = set(global_entities['keywords'] + [e[0] for e in global_entities['entities']])

    # Get main concepts from noun chunks
    main_concepts = []
    for chunk in doc.noun_chunks:
        if not all(token.is_stop for token in chunk) and len(chunk.text.split()) <= 3:
            main_concepts.append(chunk.text)

    if not main_concepts and global_concepts:
        main_concepts = list(global_concepts)[:5]

    # Process larger chunks for better context
    chunk_size = 5  # Larger chunk size for better context
    for i in range(0, len(sentences), chunk_size):
        chunk = " ".join(sentences[i:i+chunk_size])

        # Skip very short chunks
        if len(chunk.split()) < 20:
            continue

        # 1. Generate THEORY questions (50% of questions)
        theory_questions = generate_theory_questions(chunk)
        for question in theory_questions:
            questions.append((question, chunk))

        # 2. Generate T5 questions (25% of questions)
        t5_questions = generate_t5_questions(chunk, num_questions=1)
        for question in t5_questions:
            # Check if it's not a simple factual question
            if not question.lower().startswith(("what is", "who is", "when did")):
                questions.append((question, chunk))

        # 3. Generate CONCEPTUAL questions (25% of questions)
        if main_concepts:
            # Connection questions between concepts
            if len(main_concepts) >= 2:
                concept1 = random.choice(main_concepts)
                concept2 = random.choice([c for c in main_concepts if c != concept1])
                conceptual_questions = [
                    f"How does {concept1} relate to {concept2}?",
                    f"Compare and contrast {concept1} and {concept2}.",
                    f"Explain the relationship between {concept1} and {concept2} in your own words."
                ]
                questions.append((random.choice(conceptual_questions), chunk))

            # Application questions
            concept = random.choice(main_concepts)
            application_questions = [
                f"How would you apply the concept of {concept} in a real-world situation?",
                f"What are the practical implications of {concept}?",
                f"How might {concept} be used to solve a problem?"
            ]
            questions.append((random.choice(application_questions), chunk))

    # Add some high-level conceptual questions about the entire text
    if len(chapter_text.split()) > 100:
        bloom_questions = [
            "What are the core principles presented in this material?",
            "How would you synthesize the main concepts from this text?",
            "What real-world applications can you derive from these concepts?",
            "How would you explain these concepts to someone with no background in this field?",
            "What are the most significant insights you can draw from this material?"
        ]

        for q in bloom_questions[:2]:  # Add a couple higher-order questions
            questions.append((q, chapter_text))

    return questions

def compute_tfidf_similarity(user_answer, context):
    """Calculate TF-IDF based similarity between answer and context."""
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform([user_answer, context])
        return util.pytorch_cos_sim(
            torch.tensor(tfidf_matrix[0].toarray()),
            torch.tensor(tfidf_matrix[1].toarray())
        ).item()
    except:
        return 0.0

def evaluate_answer_quality(user_answer, context_text, threshold=0.6):
    """Evaluate answer based on conceptual understanding rather than text matching."""
    if not user_answer.strip():
        return False, 0.0, "No answer provided."

    # Normalize texts
    user_answer = re.sub(r'\s+', ' ', user_answer.strip().lower())
    context_text = re.sub(r'\s+', ' ', context_text.strip().lower())

    # Process with spaCy for concept extraction
    answer_doc = nlp(user_answer)
    context_doc = nlp(context_text)

    # FOCUS ON SEMANTIC SIMILARITY (60%) - most important for conceptual evaluation
    embed_answer = model.encode(user_answer, convert_to_tensor=True)
    embed_context = model.encode(context_text, convert_to_tensor=True)
    semantic_sim = util.pytorch_cos_sim(embed_answer, embed_context).item()

    # Extract concepts from both texts using improved method
    context_concepts = set()
    for chunk in context_doc.noun_chunks:
        if not all(token.is_stop for token in chunk) and len(chunk.text) > 3:
            context_concepts.add(chunk.text.lower())

    # Add entities to concepts
    for ent in context_doc.ents:
        context_concepts.add(ent.text.lower())

    answer_concepts = set()
    for chunk in answer_doc.noun_chunks:
        if not all(token.is_stop for token in chunk) and len(chunk.text) > 3:
            answer_concepts.add(chunk.text.lower())

    for ent in answer_doc.ents:
        answer_concepts.add(ent.text.lower())

    # Concept matching with approximate matching (20%)
    concept_matches = 0
    if context_concepts:
        for ans_concept in answer_concepts:
            ans_embed = model.encode(ans_concept, convert_to_tensor=True)

            # Find the best matching context concept
            max_sim = 0
            for ctx_concept in context_concepts:
                ctx_embed = model.encode(ctx_concept, convert_to_tensor=True)
                sim = util.pytorch_cos_sim(ans_embed, ctx_embed).item()
                max_sim = max(max_sim, sim)

            # Count as match if similarity is high enough
            if max_sim > 0.6:
                concept_matches += 1

        # Calculate concept coverage score
        concept_coverage = min(1.0, concept_matches / max(3, len(context_concepts) * 0.3))
    else:
        concept_coverage = 0.5  # Neutral score if no concepts found

    # Check answer quality (20%)
    # Length adequacy (not too short)
    min_expected_tokens = 15
    length_score = min(1.0, len(answer_doc) / min_expected_tokens)

    # Check for at least one complete sentence
    has_sentence = any(sent.text.strip() for sent in answer_doc.sents)
    sentence_score = 1.0 if has_sentence else 0.3

    quality_score = (0.7 * length_score + 0.3 * sentence_score)

    # Calculate final score with emphasis on meaning over matching
    final_score = (
        0.60 * semantic_sim +      # Higher weight for semantic understanding
        0.20 * concept_coverage +  # Some weight for concept coverage
        0.20 * quality_score       # Some weight for answer quality
    )

    # Dynamic threshold adjustment for question type
    # Theory questions may need more concept coverage, so make it a bit stricter
    if any(term in context_text.lower() for term in ["define", "what is", "explain the"]):
        adjusted_threshold = threshold + 0.05
    else:
        adjusted_threshold = threshold

    # Generate helpful, specific feedback
    if final_score >= adjusted_threshold:
        if semantic_sim > 0.7:
            feedback = "Excellent! Your answer shows strong conceptual understanding."
        else:
            feedback = "Good answer. You've covered the key concepts well."
    elif final_score >= adjusted_threshold * 0.7:
        feedback = "Your answer shows partial understanding of the concepts."
        if concept_coverage < 0.4:
            feedback += " Try to include more key terminology from the material."
        if semantic_sim < 0.5:
            feedback += " Make sure your explanation aligns with the core ideas in the text."
    else:
        feedback = "Your answer needs improvement in conceptual understanding."
        if concept_coverage < 0.3:
            # Provide specific concepts they missed
            missed_concepts = list(context_concepts)[:3]
            feedback += f" Consider discussing concepts like: {', '.join(missed_concepts)}."
        if quality_score < 0.4:
            feedback += " Try to provide a more complete explanation."

    return final_score >= adjusted_threshold, final_score, feedback

def extract_teaching_points(context):
    """Extract true conceptual teaching points, not just text fragments."""
    doc = nlp(context)

    # Focus on extracting real concepts
    teaching_points = []

    # 1. Look for definition sentences - these often contain key concepts
    definition_sentences = []
    for sent in doc.sents:
        sent_text = sent.text
        if is_definition_sentence(sent_text) and len(sent_text.split()) <= 25:
            definition_sentences.append(sent_text)

    # 2. Extract key terms and their contexts
    key_terms = []
    for chunk in doc.noun_chunks:
        if not all(token.is_stop for token in chunk) and 2 <= len(chunk.text.split()) <= 4:
            # Find a sentence containing this term
            for sent in doc.sents:
                if chunk.text in sent.text and len(sent.text.split()) <= 25:
                    key_terms.append(sent.text)
                    break

    # 3. Find sentences with high information density
    info_sentences = []
    for sent in doc.sents:
        # Count entities, important nouns, and verbs
        sent_doc = nlp(sent.text)
        entity_count = len(sent_doc.ents)
        noun_count = len([token for token in sent_doc if token.pos_ == "NOUN" and not token.is_stop])
        verb_count = len([token for token in sent_doc if token.pos_ == "VERB" and not token.is_stop])

        # Higher score means more informative
        info_score = entity_count + 0.5 * noun_count + 0.3 * verb_count

        if info_score >= 3 and len(sent.text.split()) <= 25:
            info_sentences.append((sent.text, info_score))

    # Sort by information score
    info_sentences.sort(key=lambda x: x[1], reverse=True)
    top_info = [s[0] for s in info_sentences[:2]]

    # Combine all sources, prioritizing definitions
    teaching_points = definition_sentences[:2] + key_terms[:2] + top_info

    # Ensure no duplicates
    unique_points = []
    for point in teaching_points:
        # Check if this point is too similar to existing ones
        if not any(util.pytorch_cos_sim(
            model.encode(point, convert_to_tensor=True),
            model.encode(existing, convert_to_tensor=True)
        ).item() > 0.8 for existing in unique_points):
            unique_points.append(point)
            if len(unique_points) >= 4:  # Limit to 4 points
                break

    # If we couldn't extract enough points, create synthetic ones
    if len(unique_points) < 2:
        # Extract key entities and concepts
        key_info = extract_key_entities(context)
        key_terms = key_info['keywords'][:3]

        for term in key_terms:
            unique_points.append(f"The concept of {term} is central to understanding this material.")

    return unique_points[:4]  # Return up to 4 teaching points

def socrateach():
    print("\n🧠 Socrateach AI Advanced - Conceptual Learning Through Questions")
    print("=" * 70)

    chapter_text = input("\n📚 Paste your chapter text here:\n")

    if len(chapter_text.split()) < 30:
        print("⚠️ Text is too short. Please provide a longer passage (at least 30 words).")
        return

    print("\n⏳ Analyzing text and generating conceptual and theory questions...")
    all_questions = generate_advanced_questions(chapter_text)

    if not all_questions:
        print("⚠️ Could not generate meaningful questions. Please try a different text.")
        return

    print(f"📝 Generated {len(all_questions)} insightful questions from your text.")

    # Filter to remove duplicates and similar questions
    filtered_questions = []
    question_embeddings = []

    for q, context in all_questions:
        q_embed = model.encode(q, convert_to_tensor=True)

        # Check if similar to existing questions
        is_duplicate = False
        for existing_embed in question_embeddings:
            similarity = util.pytorch_cos_sim(q_embed, existing_embed).item()
            if similarity > 0.7:  # Adjusted threshold for diversity
                is_duplicate = True
                break

        if not is_duplicate:
            filtered_questions.append((q, context))
            question_embeddings.append(q_embed)

    print(f"🔍 Filtered to {len(filtered_questions)} unique questions.")

    # Option to export question bank
    export = input("\nWould you like to export the complete question bank? (yes/no): ").lower()
    if export.startswith('y'):
        with open("question_bank.txt", "w", encoding="utf-8") as f:
            f.write("GENERATED QUESTION BANK\n")
            f.write("=" * 50 + "\n\n")
            for i, (q, _) in enumerate(filtered_questions):
                f.write(f"Question {i+1}: {q}\n\n")
        print("✅ Question bank exported to 'question_bank.txt'")

    print("\n🧠 Beginning your conceptual learning session...")

    # Mix of theory and conceptual questions
    # Sort to put theory questions first, then conceptual
    filtered_questions.sort(key=lambda x: 0 if any(term in x[0].lower() for term in ["what is", "define", "describe"]) else 1)

    question_count = min(len(filtered_questions), 5)
    correct_count = 0
    scores = []

    for i in range(question_count):
        question, context = filtered_questions[i]

        print(f"\n❓ Question {i+1}/{question_count}: {question}")
        user_answer = input("💬 Your answer: ").strip()

        is_correct, score, feedback = evaluate_answer_quality(user_answer, context)
        scores.append(score)

        # Provide appropriate feedback
        if is_correct:
            print(f"✅ Excellent understanding! ({score:.2f} concept mastery score)")
            print(f"📊 {feedback}")
            correct_count += 1
        elif score >= 0.5:
            print(f"🟨 Partial understanding. ({score:.2f} concept mastery score)")
            print(f"📊 {feedback}")

            teaching_points = extract_teaching_points(context)
            print("\n📌 Key concepts to reinforce:")
            for j, point in enumerate(teaching_points):
                print(f"  {j+1}. {point}")
        else:
            print(f"❌ Needs improvement. ({score:.2f} concept mastery score)")
            print(f"📊 {feedback}")

            teaching_points = extract_teaching_points(context)
            print("\n📌 Key concepts to understand:")
            for j, point in enumerate(teaching_points):
                print(f"  {j+1}. {point}")

    # Detailed session analysis
    avg_score = sum(scores) / len(scores) if scores else 0

    print("\n" + "=" * 70)
    print(f"🎓 Session complete! Overall concept mastery: {avg_score:.2f}")
    print(f"📊 You demonstrated good understanding on {correct_count} out of {question_count} questions.")

    # Personalized feedback
    if avg_score >= 0.85:
        print("🌟 Outstanding! You've demonstrated excellent conceptual mastery of the material.")
    elif avg_score >= 0.7:
        print("🎉 Great work! You have a solid grasp of the key concepts.")
    elif avg_score >= 0.5:
        print("👍 Good progress! With more focus on the core concepts, you'll master this material.")
    else:
        print("📚 This topic needs more review. Focus on understanding the fundamental concepts.")

    # Areas for improvement based on questions with lowest scores
    if scores:
        worst_q_index = scores.index(min(scores))
        worst_q, worst_context = filtered_questions[worst_q_index]

        print(f"\n📝 Concept to focus on: '{worst_q}'")
        teaching_points = extract_teaching_points(worst_context)
        print("Review these key points:")
        for j, point in enumerate(teaching_points):
            print(f"  {j+1}. {point}")

    # Ask if they want to continue
    if len(filtered_questions) > question_count:
        more = input("\nWould you like to continue with more questions? (yes/no): ").lower()
        if more.startswith('y'):
            print("\nContinuing with more questions...\n")
            remaining_questions = filtered_questions[question_count:]
            filtered_questions = remaining_questions
            # Continue with the next batch
            socrateach()
    else:
        print("\n✨ You've completed all available questions for this text!")
        print("📚 For more practice, try with a different section of your study materials.")

if __name__ == "__main__":
    socrateach()


🧠 Socrateach AI Advanced - Conceptual Learning Through Questions
